#### CSCE 670 :: Information Storage and Retrieval :: Texas A&M University :: Spring 2020


# Homework 3:   Recommender System Practice: Rating Prediction and Top-K Item Recommendation

### 100 points [ 6% of your final grade]

### Due: April 10, 2020

*Goals of this homework:* Understand matrix factorization (MF) using explicit feedback and Bayesian Personalized Ranking (BPR) using implicit feedback for recommendation. Explore different methods for two real-world recommendation senarios: rating prediction and top-K item recommendation.

*Submission instructions (eCampus):* To submit your homework, rename this notebook as `UIN_hw3.ipynb`. For example, my homework submission would be something like `555001234_hw3.ipynb`. Submit this notebook via eCampus (look for the homework 3 assignment there). Your notebook should be completely self-contained, with the results visible in the notebook. We should not have to run any code from the command line, nor should we have to run your code within the notebook (though we reserve the right to do so). So please run all the cells for us, and then submit.

*Late submission policy:* For this homework, you may use as many late days as you like (up to the total late days you have remaining).

*Collaboration policy:* You are expected to complete each homework independently. Your solution should be written by you without the direct aid or help of anyone else. However, we believe that collaboration and team work are important for facilitating learning, so we encourage you to discuss problems and general problem approaches (but not actual solutions) with your classmates. You may post on Piazza, search StackOverflow, etc. But if you do get help in this way, you must inform us by **filling out the Collaboration Declarations at the bottom of this notebook**. 

*Example: I found helpful code on stackoverflow at https://stackoverflow.com/questions/11764539/writing-fizzbuzz that helped me solve Problem 2.*

The basic rule is that no student should explicitly share a solution with another student (and thereby circumvent the basic learning process), but it is okay to share general approaches, directions, and so on. If you feel like you have an issue that needs clarification, feel free to contact either me or the TA.

# Part 1. Matrix Factorization for Rating Prediction (70 points total)

In some platforms, such as MovieLens, users express their preference on items using explict feedback like ratings.

In this part, you will implement matrix factorization to predict ratings on MovieLens data. After removing users who left less than 20 ratings and movies with less than 20 ratings, the provided dataset has only ~1,200 items and ~500 users. You can also check the title and genres of each movie in *movies_info.csv*.

## Part 1a: Load the Data (5 points)

Please download the dataset from Piazza. There are about 65,000 ratings in total. We split the rating data into two sets. You will train with 70% of the data (in *train_movie.csv*) and test on the remaining 30% of data (in *test_movie.csv*). Each of train and test files has lines having this format: UserID, MovieID, Rating. 

First you will need to load the data and store it with any structure you like. Please report the numbers of unique users and movies in the dataset. 

In [1]:
# load the data, then print out the number of
# movies and users in each of train and test sets.
# Your Code Here...
import pandas as pd
import numpy as np

train_data = pd.read_csv("./rating prediction dataset/train_movie.csv")
train_data.head()

users = train_data['userId'].unique()
movies = train_data['movieId'].unique()

num_users = len(users)
num_movies = len(movies)

print("num_users:", num_users)
print("num_movies:",num_movies)

uid = dict(zip(users,range(0,num_users)))
mid = dict(zip(movies,range(0,num_movies)))

train_data = train_data.sample(frac=1)
train_np = train_data.values
np.random.shuffle(train_np)

train_samples = int(train_data.shape[0]*0.8)
train_80 = train_np[:train_samples,:]
cross_20 = train_np[train_samples:,:]

mat= np.full((num_users,num_movies),-1)

# train data numpy matris
td = train_80

for i in range(0,td.shape[0]):
    mat[uid[td[i,0]],mid[td[i,1]]] = td[i,2]

num_users: 541
num_movies: 1211


## Part 1b: Matrix Factorization (40 points)

In class, we introduced how matrix factorization works for recommendation. Now it is your term to implement it. There are different methods to obtain the latent factor matrices **P** and **Q**, like gradient descent, Alternating Least Squares (ALS), and so on. Pick one of them and implement your MF model. *You can refer to tutorials and resources online. Remember our **collaboration policy** and you need to inform us of the resources you refer to.* 

Please report MAE and RMSE of your MF model for the test set.

In [2]:
# Your Code Here...
# Report Mean Absolute Error and Root Mean Squared Error for test

class MF:
    def __init__(self, train, hidden=40, epoch=10, lr=0.01, bias=False, r = 0.01):
        self.bias = bias
        self.train = train
        self.mask = 1.0 * (train >= 0)
        self.num_train = np.sum(self.mask)
        self.num_user, self.num_item = train.shape
        self.hidden = hidden
        self.regularization = r

        if self.bias:
            self.global_bias = np.sum(train) / np.sum(self.mask)
            self.user_bias = np.sum((train - self.global_bias) * self.mask, axis=1) / np.sum(self.mask, axis=1)
            self.item_bias = np.sum((train - self.global_bias) * self.mask, axis=0) / np.sum(self.mask, axis=0)
        else:
            self.user_bias = np.zeros(self.num_user)
            self.item_bias = np.zeros(self.num_item)
            self.global_bias = 0.0

        self.epoch = epoch

        self.sample_row, self.sample_col = np.where(train>=0)
        self.n_samples = len(self.sample_row)
        print(self.n_samples)
        self.lr = lr


    def fit(self):
        # initialize
        self.P = np.random.random((self.num_user, self.hidden))
        self.Q = np.random.random((self.num_item, self.hidden))

        for itr in range(self.epoch):
            self.training_indices = np.arange(self.n_samples)
            np.random.shuffle(self.training_indices)
            for idx in self.training_indices:
                u = self.sample_row[idx]
                i = self.sample_col[idx]
                rating = self.train[u, i]
                e = rating - self.P[u, :].dot(self.Q[i, :].T) - self.global_bias - self.item_bias[i] - self.user_bias[u]
                self.P[u, :] += self.lr * (e * self.Q[i, :] - self.regularization * self.P[u, :])
                self.Q[i, :] += self.lr * (e * self.P[u, :] - self.regularization * self.Q[u, :])

            Rec = self.P.dot(self.Q.T) + self.global_bias + self.item_bias + self.user_bias.reshape((-1, 1))
            error = np.sqrt(np.sum(np.square(Rec * self.mask - self.train * self.mask)) / self.num_train)
            if itr % 10 == 0:
                print('iters={0}, RMSE={1}'.format(itr, error))
        return self.P, self.Q, (self.P.dot(self.Q.T) + self.global_bias + self.item_bias + self.user_bias.reshape((-1, 1)))

In [3]:
import hyperopt
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from hyperopt import space_eval

# We want to minimize loss i.e. negative of accuracy
def optimize(hyperparam):
    print(hyperparam)

    mf = MF(mat, hidden=hyperparam['hiddens'], lr=hyperparam['lrs'], epoch=25,r = hyperparam['regs'])
    P_mf, Q_mf, Rec_mf = mf.fit()
    
    RMSE = 0

    for row in range(cross_20.shape[0]):
        score = P_mf[uid[cross_20[row,0]],:].dot(np.transpose(Q_mf[mid[cross_20[row,1]],:]))
        RMSE += np.square(cross_20[row,2]-score)

    RMSE = RMSE/cross_20.shape[0]
    RMSE = np.sqrt(RMSE)
    
    print("RMSE: ",RMSE)
    
    return({"status": STATUS_OK, "loss": RMSE})


In [4]:

# Define search space for hyper-parameters
space = {
    'hiddens': hp.choice('hiddens', [5,8,10,12]),
    'lrs': hp.choice('lrs', [0.001,0.003,0.01]),
    'regs': hp.choice('regs', [0.01,0.05,0.1,0.5]),
}

trials_mf = Trials()

# Find the best hyperparameters
best = fmin(
        optimize,
        space,
        algo=tpe.suggest,
        trials=trials_mf,
        max_evals=10,
    )

{'hiddens': 12, 'lrs': 0.003, 'regs': 0.1}                                                                             
36225                                                                                                                  
iters=0, RMSE=1.0695564737218985                                                                                       
iters=10, RMSE=0.8465132335779115                                                                                      
iters=20, RMSE=0.8227690202137479                                                                                      
RMSE:                                                                                                                  
0.9093528831374047                                                                                                     
{'hiddens': 12, 'lrs': 0.003, 'regs': 0.5}                                                                             
36225                                   

In [5]:
td_final = train_data.values
mat_train= np.full((num_users,num_movies),-1)


for i in range(0,td_final.shape[0]):
    mat_train[uid[td_final[i,0]],mid[td_final[i,1]]] = td_final[i,2]
    
# best hyperparamters found by hyperparamter tuning 

best_params = space_eval(space,best)

mf = MF(mat_train, hidden=best_params['hiddens'], lr=best_params['lrs'], epoch=100,r = best_params['regs'])
P_mf, Q_mf, Rec_mf = mf.fit()

45282
iters=0, RMSE=1.0971005329751187
iters=10, RMSE=0.8495275012336072
iters=20, RMSE=0.8341123325234236
iters=30, RMSE=0.8212619901816604
iters=40, RMSE=0.8074849482808698
iters=50, RMSE=0.7919864123693288
iters=60, RMSE=0.7757768800440233
iters=70, RMSE=0.7611068410448105
iters=80, RMSE=0.7467851450938148
iters=90, RMSE=0.7345240942228494


In [6]:
test_data = pd.read_csv("./rating prediction dataset/test_movie.csv")
test_data.head()

MAE = 0
RMSE = 0

y_pred_mf = np.zeros(test_data.shape[0])

idx = 0
for row in test_data.itertuples(index=False):
    score = P_mf[uid[row[0]],:].dot(np.transpose(Q_mf[mid[row[1]],:]))
    y_pred_mf[idx] = score
    idx +=1
    MAE += abs(row[2]-score)
    RMSE += np.square(row[2]-score)

MAE = MAE/test_data.shape[0]
RMSE = RMSE/test_data.shape[0]
RMSE = np.sqrt(RMSE)

print(MAE)
print(RMSE)

0.7083750094272993
0.9173159917603063


#### MAE = 0.7083
#### RMSE = 0.9173

Which method did you use to obtain **P** and **Q**? What are the advantages and disadvantages of the method you pick? *provide a brief (1-2 paragraph) discussion based on these questions.*

I have used stochastic gradient descent for matrix factorization. My inclination towards SGD was because of sources suggesting better results using it [Here: http://cs229.stanford.edu/proj2014/Christopher%20Aberger,%20Recommender.pdf]

The improved accuracy however comes at the expanse of training time which is much more compared to ALS. SGD also has a tendency to overfit much and hence a wider hyperparamter space needs to be searched for. Which further worsens the already slow compute time exponentially.

## Part 1c: Improve MF (25 points)

Given your results in the previous part, can you do better? For this last part you should report on your best attempt at improving MAE and RMSE. Provide code, results, plus a brief discussion on your approach. Hints: You may consider using the title or genres information, trying other algorithms, designing a hybrid system or considering a neighborhood like this paper [Factorization Meets the Neighborhood: a Multifaceted Collaborative Filtering Model](https://www.cs.rochester.edu/twiki/pub/Main/HarpSeminar/Factorization_Meets_the_Neighborhood-_a_Multifaceted_Collaborative_Filtering_Model.pdf). *You can do anything you like to improve MAE and RMSE.*

You will get full marks for this part if you get better results than your MF results (of course we will also judge whether what you do here is reasonable or not). You will get partial marks for a reasonable effort even if you do not improve your MF results. Additionally, you will get 5 points as bonus if your model performs the best among the whole class.

In [7]:
# lets get them movie categories shall we?
from itertools import chain

file1 = open("./rating prediction dataset/movies_info.csv", 'r') 
Lines = file1.readlines() 

categories = set()
genre = {}
year = {}

count = 0
for line in Lines:
    if count == 0:
        count = 1
        continue

    mov = line.split("	")
    if len(mov)>1:
        mov[2] = mov[2].strip()
        genre[int(mov[0])] = mov[2].split("|")
        year[int(mov[0])] = int(mov[1][-5:-1])


categories = set(chain(*genre.values()))

cat = {}
idx = 0
for category in categories:
    cat[category] = idx
    idx += 1

In [8]:
# Your Code Here...
# Report Mean Absolute Error and Root Mean Squared Error for test

# let us create training and testing datasets from the user vectors and movie vectors
factors = P_mf.shape[1]
features = factors*2 + 22

x_train = np.zeros((train_data.shape[0],features))
y_train = np.zeros((train_data.shape[0],))

for i in range(train_data.shape[0]):
    _user = uid[train_data.values[i,0]]
    _movie = mid[train_data.values[i,1]]
    x_train[i,:factors] = P_mf[_user,:]
    x_train[i,factors:factors*2] = Q_mf[_movie,:]
    x_train[i,-2] = year[_movie]
    x_train[i,-1] = P_mf[_user,:].dot(np.transpose(Q_mf[_movie,:]))

    for c in genre[_movie]:
        x_train[i,factors*2+cat[c]] = 1

    y_train[i] = train_data.values[i,2]

    
x_test = np.zeros((test_data.shape[0],features))
y_test = np.zeros((test_data.shape[0],))

for i in range(test_data.shape[0]):
    _user = uid[test_data.values[i,0]]
    _movie = mid[test_data.values[i,1]]
    x_test[i,:factors] = P_mf[_user,:]
    x_test[i,factors:factors*2] = Q_mf[_movie,:]
    x_test[i,-2] = year[_movie]
    x_test[i,-1] = P_mf[_user,:].dot(np.transpose(Q_mf[_movie,:]))
    
    for c in genre[_movie]:
        x_test[i,factors*2+cat[c]] = 1

    y_test[i] = test_data.values[i,2]


In [9]:
# fit model no training data
from xgboost import XGBRegressor

def optimize_xgb(hyperparam):
    print(hyperparam)

    split_at = int(x_train.shape[0] * 0.8)
    opt_x_train = x_train[:split_at,:]
    opt_y_train = y_train[:split_at]
    
    opt_x_val = x_train[split_at:,:]
    opt_y_val = y_train[split_at:]
    
    model = XGBRegressor(**hyperparam)
    model.fit(opt_x_train,opt_y_train)
    
    opt_y_pred = model.predict(opt_x_val)
    
    opt_rmse = np.sqrt(np.mean(np.square(opt_y_pred-opt_y_val)))

    print("RMSE:",opt_rmse)
    
    return({"status": STATUS_OK, "loss": opt_rmse})

In [10]:
# XGB parameters
space_xgb = {
    'learning_rate':    hp.choice('learning_rate',    np.arange(0.05, 0.31, 0.05)),
    'max_depth':        hp.choice('max_depth',        np.arange(5, 16, 1, dtype=int)),
    'min_child_weight': hp.choice('min_child_weight', np.arange(1, 8, 1, dtype=int)),
    'colsample_bytree': hp.choice('colsample_bytree', np.arange(0.3, 0.8, 0.1)),
    'subsample':        hp.uniform('subsample', 0.8, 1),
    'n_estimators':     100,
}

trials_xgb = Trials()

# Find the best hyperparameters
best_xgb = fmin(
        optimize_xgb,
        space_xgb,
        algo=tpe.suggest,
        trials=trials_xgb,
        max_evals=5,
    )

{'colsample_bytree': 0.6000000000000001, 'learning_rate': 0.1, 'max_depth': 12, 'min_child_weight': 7, 'n_estimators': 100, 'subsample': 0.8569738004137474}
[22:50:43] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
RMSE:                                                                                                                  
0.7477775527082361                                                                                                     
{'colsample_bytree': 0.6000000000000001, 'learning_rate': 0.15000000000000002, 'max_depth': 9, 'min_child_weight': 7, 'n_estimators': 100, 'subsample': 0.9575782580855137}
[22:51:21] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
RMSE:                                                                                                  

In [11]:
model_opt = XGBRegressor(colsample_bytree= 0.7, learning_rate= 0.1, max_depth= 7, min_child_weight= 7, n_estimators= 100, subsample= 0.97)
model_opt.fit(x_train,y_train)

y_pred = model_opt.predict(x_test)

rmse = np.sqrt(np.mean(np.square(y_pred-y_test)))
mae = np.mean(np.abs(y_pred-y_test))
print("RMSE:",rmse)
print("MAE:",mae)

[22:53:18] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
RMSE: 0.9398960787812505
MAE: 0.7116150245555766


In [12]:
y_pred_2 = y_pred + y_pred_mf
y_pred_2 /= 2
rmse = np.sqrt(np.mean(np.square(y_pred_2-y_test)))
mae = np.mean(np.abs(y_pred_2-y_test))
print("MAE:",mae)
print("RMSE:",rmse)

MAE: 0.7057971994236139
RMSE: 0.9234126901026305


### Attempt 2:

In [13]:
# lets try ensemble methods
import surprise
from sklearn.model_selection import train_test_split

df_train = pd.read_csv("./rating prediction dataset/train_movie.csv")

rawTrain,rawholdout = train_test_split(df_train, test_size=0.25)
reader = surprise.Reader(rating_scale=(1,5)) 

#into surprise:
data = surprise.Dataset.load_from_df(rawTrain,reader)
holdout = surprise.Dataset.load_from_df(rawholdout,reader)

trainset = data.build_full_trainset()
testset = holdout.build_full_trainset().build_testset()

In [14]:
x_train = np.zeros((rawholdout.shape[0],4))
y_train = np.zeros(rawholdout.shape[0])

i=0
for test in testset:
    y_train[i]=test[2]
    i += 1

In [15]:
sim_options = sim_options = {'name': 'cosine',
               'user_based': False  # compute  similarities between items
               }
collabKNN = surprise.KNNWithMeans(k=40,sim_options=sim_options) #try removing sim_options. You'll find memory errors. 

collabKNN.fit(trainset)
predictionsKNN = collabKNN.test(testset)
surprise.accuracy.rmse(predictionsKNN,verbose=True)

i = 0
for prediction in predictionsKNN:
    x_train[i,0] = prediction.est
    i += 1

Computing the cosine similarity matrix...


C:\Users\shubh\Anaconda3\lib\site-packages\surprise\prediction_algorithms\algo_base.py:248: RuntimeWarning: invalid value encountered in double_scalars
  sim = construction_func[name](*args)


Done computing similarity matrix.
RMSE: 0.8999


In [16]:
funkSVD = surprise.prediction_algorithms.matrix_factorization.SVD(n_factors=50,n_epochs=30,biased=True)
funkSVD.fit(trainset)
predictionsSVD = funkSVD.test(testset)
surprise.accuracy.rmse(predictionsSVD,verbose=True)

i = 0
for prediction in predictionsSVD:
    x_train[i,1] = prediction.est
    i += 1

RMSE: 0.8946


In [17]:
coClus = surprise.prediction_algorithms.co_clustering.CoClustering(n_cltr_u=4,n_cltr_i=4,n_epochs=25) 
coClus.fit(trainset)
predictionsCoClus = coClus.test(testset)
surprise.accuracy.rmse(predictionsCoClus,verbose=True)

i = 0
for prediction in predictionsCoClus:
    x_train[i,2] = prediction.est
    i += 1

RMSE: 0.9226


In [18]:
slopeOne = surprise.prediction_algorithms.slope_one.SlopeOne()
slopeOne.fit(trainset)
predictionsSlope = slopeOne.test(testset)
surprise.accuracy.rmse(predictionsSlope,verbose=True)

i = 0
for prediction in predictionsSlope:
    x_train[i,3] = prediction.est
    i += 1

RMSE: 0.8980


In [19]:
from sklearn import linear_model

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(x_train, y_train)

# Make predictions using the testing set
y_pred = regr.predict(x_train)


In [20]:
test_data = pd.read_csv("./rating prediction dataset/test_movie.csv")
test = surprise.Dataset.load_from_df(test_data,reader)

TEST_trainset = test.build_full_trainset()
TEST_testset = TEST_trainset.build_testset()

x_test = np.zeros((test_data.shape[0],4))
y_test = np.zeros(test_data.shape[0])

i=0
for test in TEST_testset:
    y_test[i]=test[2]
    i += 1

predictionsKNN = collabKNN.test(TEST_testset)

i = 0
for prediction in predictionsKNN:
    x_test[i,0] = prediction.est
    i += 1

predictionsSVD = funkSVD.test(TEST_testset)

i = 0
for prediction in predictionsSVD:
    x_test[i,1] = prediction.est
    i += 1

predictionsCoClus = coClus.test(TEST_testset)

i = 0
for prediction in predictionsCoClus:
    x_test[i,2] = prediction.est
    i += 1

predictionsSlope = slopeOne.test(TEST_testset)

i = 0
for prediction in predictionsSlope:
    x_test[i,3] = prediction.est
    i += 1

# Make predictions using the testing set
y_pred = regr.predict(x_test)

In [21]:
rmse = np.sqrt(np.mean(np.square(y_pred-y_test)))
mae = np.mean(np.abs(y_pred-y_test))
print("RMSE:",rmse)
print("MAE:",mae)

RMSE: 0.8799290817017825
MAE: 0.6773006644638268


Please explain what you do to improve the recommendation in 1-2 paragraphs.

#### Attempt 1 - Using Content information with features
I used the movie release year and genre as features alongside the user and movie feature vectors from the matrix factorization algorithm in a random forest regression model. I did not see any imporvement in that model though.


#### Attempt 2 - Ensemble ratings from multiple models (KNN + SVD + Clustering + slope one)
I used 4 different models and learn predictions from that. These predictions are then used to train a linear regression model which will learn parameters giving a weighted average.

#### MAE: 0.6773006644638268 
(using mf: 0.7083750094272993)
#### RMSE: 0.8799290817017825 
(using mf: 0.9173159917603063)


# Part 2. Bayesian Personalized Ranking (BPR) for Top-K Item Recommendation (30 points)

Compared with rating prediction in part 1, a more popular scenario recently is personalized top-K item ranking for each user based on the user's implicit feedback. Examples include ranking videos on YouTube and ranking products on Aamzon. In practice, users tend to provide implicit feedback (e.g., the user clicked a product URL on Amazon or played a video on YouTube) rather than explicit feedback (e.g., ratings or reviews) in most cases.

In this part, you will experiment with Bayesian Personalized Ranking (BPR) to rank items on a [Spotify Playlist Recommendation Dataset](http://people.tamu.edu/~yunhe/pubs/AttListCIKM2019.pdf). If a user ever followed a playlist, this interaction is treated as an implicit feedback. In our sampled dataset, there are ~10,000 users and ~7,000 playlists.

BPR can generate scores of items for each user. You should rank all items based on the scores for each user and evaluate the ranking performance.

For example, if user 0 has two interacted playlists 23, 78 in test.txt. If the top-10 playlists for user 0 returned by BPR is [12,45,78,34,23,90,134,33,46,9], then the precision@10 for user 0 is 0.2 because the two playlists in test.txt are recommended in top-10: 2/10=0.2. Please report NDCG@10 in this part.

## Load the Data

Please download the dataset from Piazza. There are about 90,000 interactions in total, which are split into training.txt, validation.txt and text.txt. You will train on train.txt, tune hyperparameters on validation.txt and report final result on test.txt in terms of NDCG@10. 

Each of the train and test files has lines having this format: UserID, PlaylistID, 1.0. 

First you will need to load the data and store it with any structure you like. Please report the numbers of unique users and movies in the dataset. 

In [22]:
# load the data, then print out the number of
# playlists and users in each of train and test sets.
# Your Code Here...
from collections import defaultdict 
from implicit import bpr
import scipy.sparse as sp

df_train = pd.read_csv("./top-K recommendation dataset/train.txt",sep="\t",header=None)

users = df_train[0].unique()
plays = df_train[1].unique()

num_users = len(users)
num_plays = len(plays)

print("Number of unique users:", num_users)
print("Number of unique playlists:",num_plays)

df_train = df_train.sample(frac=1)

mat_bpr = np.full((num_users,num_plays),0)

# train data numpy matris
td = df_train.values
td = td.astype(int)

for i in range(0,td.shape[0]):
    mat_bpr[td[i,0],td[i,1]] = td[i,2]


# sparse matrix for bpr in implicit package
mat_coo = sp.csr_matrix(mat_bpr.T)

Number of unique users: 10183
Number of unique playlists: 7787


In [23]:
df_val = pd.read_csv("./top-K recommendation dataset/validation.txt",sep="\t",header=None)
df_test = pd.read_csv("./top-K recommendation dataset/test.txt",sep="\t",header=None)


recommend_val  = defaultdict(list)

for row in df_val.itertuples():
    recommend_val[row[1]].append(row[2])
    
recommend_test = defaultdict(list)

for row in df_test.itertuples():
    recommend_test[row[1]].append(row[2])

## BPR by Using Package

Compared with MF, BPR is more complicated to implement. In this part, you can use a BPR package to experiment with top-K item recommendation. Some good packages include https://github.com/benfred/implicit.

In [24]:
import math
def ndcg(topk,match):
    rel = []
    for k in topk:
        if k in match:
            rel.append(1)
        else:
            rel.append(0)

    dcg = 0
    for i in range(1,11):
        dcg += rel[i-1]/math.log(1+i)
    
    idcg = 0
    rel = sorted(rel,reverse=True)
    for i in range(1,11):
        idcg += rel[i-1]/math.log(1+i)
    
    return dcg/(idcg+1e-10)

In [25]:
# your code to call other BPR packages for top-K recommendation.
# Report average NDCG@10 for all users on test.txt

def optimize_bpr(hyperparam):
    print(hyperparam)
    
    model2 = bpr.BayesianPersonalizedRanking(**hyperparam)
    model2.fit(mat_coo)    

    avg_ndcg = 0
    count = 0
    
    for user,playlists in recommend_val.items():
        rec = model2.recommend(user,mat_coo.T.tocsr(),10)
        rec = [i[0] for i in rec]
        avg_ndcg += ndcg(rec,playlists)
        count += 1

    avg_ndcg/= count
    
    print("NDCG: ",avg_ndcg)
    
    return({"status": STATUS_OK, "loss": -1 * avg_ndcg})

In [26]:
# XGB parameters
space_bpr = {
    'factors':    hp.choice('factors ', [50,100]),
    'learning_rate':        hp.choice('learning_rate', [0.01,0.03]),
    'regularization': hp.choice('regularization', [0.005,0.01,0.05]),
}

trials_bpr = Trials()

# Find the best hyperparameters
best_bpr = fmin(
        optimize_bpr,
        space_bpr,
        algo=tpe.suggest,
        trials=trials_bpr,
        max_evals=10,
    )

{'factors': 50, 'learning_rate': 0.03, 'regularization': 0.05}                                                         
  0%|                                                                           | 0/10 [00:00<?, ?trial/s, best loss=?]

NDCG:                                                                                                                  
0.10973048281755975                                                                                                    
{'factors': 100, 'learning_rate': 0.01, 'regularization': 0.05}                                                        
 10%|████▋                                          | 1/10 [00:15<02:16, 15.19s/trial, best loss: -0.10973048281755975]

NDCG:                                                                                                                  
0.05852117250036404                                                                                                    
{'factors': 50, 'learning_rate': 0.01, 'regularization': 0.005}                                                        
 20%|█████████▍                                     | 2/10 [00:32<02:06, 15.77s/trial, best loss: -0.10973048281755975]

NDCG:                                                                                                                  
0.12420095848448141                                                                                                    
{'factors': 50, 'learning_rate': 0.01, 'regularization': 0.05}                                                         
 30%|██████████████                                 | 3/10 [00:49<01:53, 16.19s/trial, best loss: -0.12420095848448141]

NDCG:                                                                                                                  
0.06800318527016509                                                                                                    
{'factors': 100, 'learning_rate': 0.03, 'regularization': 0.005}                                                       
 40%|██████████████████▊                            | 4/10 [01:06<01:38, 16.36s/trial, best loss: -0.12420095848448141]

...

NDCG:                                                                                                                  
0.124900558398552                                                                                                      
{'factors': 100, 'learning_rate': 0.01, 'regularization': 0.05}                                                        
 60%|████████████████████████████▏                  | 6/10 [01:39<01:06, 16.52s/trial, best loss: -0.12682503764145006]

NDCG:                                                                                                                  
0.05483024104878363                                                                                                    
{'factors': 50, 'learning_rate': 0.03, 'regularization': 0.01}                                                         
 70%|████████████████████████████████▉              | 7/10 [01:56<00:49, 16.58s/trial, best loss: -0.12682503764145006]

NDCG:                                                                                                                  
0.12314794797990702                                                                                                    
{'factors': 100, 'learning_rate': 0.01, 'regularization': 0.005}                                                       
 80%|█████████████████████████████████████▌         | 8/10 [02:12<00:33, 16.51s/trial, best loss: -0.12682503764145006]

NDCG:                                                                                                                  
0.12205634255748887                                                                                                    
{'factors': 50, 'learning_rate': 0.03, 'regularization': 0.005}                                                        
 90%|██████████████████████████████████████████▎    | 9/10 [02:29<00:16, 16.54s/trial, best loss: -0.12682503764145006]

NDCG:                                                                                                                  
0.1205188646607641                                                                                                     
100%|██████████████████████████████████████████████| 10/10 [02:46<00:00, 16.59s/trial, best loss: -0.12682503764145006]


In [27]:
bpr_hyper = space_eval(space_bpr, best_bpr)
print("Best hyperparameters for bpr:")
print(bpr_hyper)

Best hyperparameters for bpr:
{'factors': 100, 'learning_rate': 0.03, 'regularization': 0.005}


In [28]:
# we now have the best parameters
# add validation to our training data

for user in recommend_val:
    for playlist in recommend_val[user]:
        mat_bpr[user,playlist] = 1

# sparse matrix for bpr in implicit package
mat_coo = sp.csr_matrix(mat_bpr.T)

In [29]:
#train and report testing NDCG

model_final = bpr.BayesianPersonalizedRanking(**bpr_hyper)
model_final.fit(mat_coo)    

avg_ndcg = 0
count = 0

for user,playlists in recommend_test.items():
    rec = model_final.recommend(user,mat_coo.T.tocsr(),10)
    rec = [i[0] for i in rec]
    avg_ndcg += ndcg(rec,playlists)
    count += 1

avg_ndcg/= count

print("Test NDCG: ",avg_ndcg)



Test NDCG:  0.12712006783311303


### Test NDCG:    0.1271

## Collaboration declarations

*If you collaborated with anyone (see Collaboration policy at the top of this homework), you can put your collaboration declarations here.*

1. Imported recdemo MF and made modifictions